In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from tqdm import tqdm
from sklearn.neighbors import KDTree

In [2]:
# path to data
raw_data_path = "/home/ctchervenkov/Documents/data/ARE/are_npvm_referenz_2040_data/"
output_path = "/home/ctchervenkov/Documents/data/ARE/"
shapefile = "05-DTV_2040_OEV_Referenz_KGM_1250_reduziert/DTV_2040_OEV_Referenz__KGM_1250_reduziert_link.SHP";

In [3]:
# read private transport data
df_raw = gpd.read_file(
            "%s/%s" % (raw_data_path, shapefile),
            encoding = "latin1"
        ).to_crs({'init': 'EPSG:2056'})

In [4]:
# list of fields and renames
fields_direction_A = ['NO', 'FROMNODENO', 'TONODENO', 'LENGTH', 'TYPENO', 'TSYSSET','VOLPERSP~1']

fields_direction_B = ['R_NO', 'R_FROMNO~2', 'R_TONODENO', 'R_LENGTH', 'R_TYPENO', 'R_TSYSSET', 'R_VOLPER~3']

renames = {
    'NO' : 'link_id',
    'FROMNODENO' : 'from_node',
    'TONODENO' : 'to_node',
    'LENGTH' : 'link_length_km',
    'TYPENO' : 'type_id',
    'TSYSSET' : 'public_transit_system',
    'VOLPERSP~1' : 'person/day',
    'R_NO' : 'link_id',
    'R_FROMNO~2' : 'from_node',
    'R_TONODENO' : 'to_node',
    'R_LENGTH' : 'link_length_km',
    'R_TYPENO' : 'type_id',
    'R_TSYSSET' : 'public_transit_system',
    'R_VOLPER~3' : 'person/day',
    'geometry' : 'geometry',
}

In [5]:
# separate by direction
df_raw_direction_A = df_raw[fields_direction_A].rename(index=str, columns=renames)
df_raw_direction_B = df_raw[fields_direction_B].rename(index=str, columns=renames)

In [6]:
# clean data
df_raw_direction_A.loc[:,"direction"] = 0
df_raw_direction_B.loc[:,"direction"] = 1
df_raw_direction_A["link_length_km"] = df_raw_direction_A["link_length_km"].str.split("km").str.get(0).astype(float)
df_raw_direction_B["link_length_km"] = df_raw_direction_B["link_length_km"].str.split("km").str.get(0).astype(float)

In [7]:
# merge
df = pd.concat([df_raw_direction_A, df_raw_direction_B])

In [8]:
# compute travelled km
df.loc[:, "person_km/day"] = df.loc[:, "person/day"] * df.loc[:, "link_length_km"]

/home/ctchervenkov/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [9]:
df = df[["person_km/day", "person/day", "direction"]].groupby("direction").sum()

In [10]:
df.to_csv("%s/%s" % (output_path, "oev_statistics.csv"), sep=";")